In [1]:
import qiskit as qk
import numpy as np
import qiskit.quantum_info as qi

from qiskit import Aer

In [2]:
backend = Aer.get_backend('qasm_simulator')

nqubits=2

Consider the observable $O$ and two generic quantum states $\ket{\psi}$ and $\ket{\phi}$. We want to calculate with quantum hardware the quantity
$$
\bra{\psi}P\ket{\phi}.
$$
where $P$ is a Pauli operator.

First of all we shall prepare the states $\ket{\psi}$ and $\ket{\phi}$ using a quantum circuit for each of them. So we actually have
$$
\ket{\psi} = U_{\psi}\ket{0} \qquad \ket{\phi} = U_{\phi}\ket{0}
$$

In [3]:
def Upsi(nqubits):
    '''Prepare the state $\psi$ as you prefer'''
    qc = qk.QuantumCircuit(nqubits)

    # customize this part
    qc.h(1)
    #
    
    return qc


In [4]:
psi = qi.Statevector.from_instruction(Upsi(nqubits))

In [5]:
def Uphi(nqubits):
    '''Prepare the state $\phi$ as you prefer'''
    qc = qk.QuantumCircuit(nqubits)

    # customize this part
    qc.x(0)
    #
    
    return qc

In [6]:
phi = qi.Statevector.from_instruction(Uphi(nqubits))


Let's define an observable we want to use:
$$
O = X_1X_2
$$

In [7]:
P = qi.Operator(qi.Pauli('XX'))

P.data.real

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

Now we can evaluate the matrix element using the following fact:
$$
\bra{\psi}O\ket{\phi} = \bra{0}U_\psi^\dag P U_\phi\ket{0}
$$
This is just an expectation value which can be solved with a simple Hadamard test.

In [8]:
# Analytical derivation
mel_expected = phi.evolve(P).inner(psi).real

In [12]:
# Quantum Hardware derivation
def mel(qc_psi, qc_phi, P, shots=10000):
    qr = qk.QuantumRegister(nqubits+1)
    cr = qk.ClassicalRegister(1)
    qc_main = qk.QuantumCircuit(qr, cr)

    # get the unitaries out of psi and phi definitions
    U_psi = qk.extensions.UnitaryGate(qi.Operator(qc_psi))
    U_phi = qk.extensions.UnitaryGate(qi.Operator(qc_phi))

    # Initialize a temp circuit to convert (Upsi P Uphi) into a controlled gate
    qc_temp = qk.QuantumCircuit(nqubits, name="VPU")
    qc_temp.append(U_phi, list(range(nqubits)))
    qc_temp.append(P, list(range(nqubits)))
    qc_temp.append(U_psi.adjoint(), list(range(nqubits)))
        # make it controlled
    VPU = qc_temp.to_gate().control(1)

    # Perform hadamard test
    qc_main.h(0)
    qc_main.append(VPU, list(range(nqubits+1)))
    qc_main.h(0)
    qc_main.measure(qr[0], cr[0])

    counts = qk.execute(qc_main, backend, shots=shots).result().get_counts()
    if len(counts)==1:
        try:
            counts['0']
            mean_val = 1
        except:
            mean_val = -1
    else:
        mean_val = (counts['0']-counts['1'])/shots
        #from binomial
        error = np.sqrt(2*counts['0']*counts['1']/shots)/shots

    return mean_val, error

    

In [13]:
shots = 10000
mel_observed, mel_error = mel(Upsi(nqubits), Uphi(nqubits), P, shots)

In [14]:
print("Analytical value:\t", mel_expected)
print("Observed value:\t\t", mel_observed, ' +- ', mel_error)

Analytical value:	 0.7071067811865475
Observed value:		 0.7118  +-  0.004966592191835363
